In [1]:
import pybamm
import numpy as np

In [2]:
options = {"thermal": "x-full"}
model = pybamm.lithium_ion.SPMe(options=options) 
parameter_values = model.default_parameter_values

In [4]:
# defining array of C_rates
C_Rates = [0.1, 0.2, 0.4, 0.8, 1, 2, 4, 6, 8, 10]


NSOC = []
Temp = []
Voltage = []
C_Rate = []

SPMT_RMSE = []

n = len(C_Rates)


for i in range (n):
    sim = pybamm.Simulation(model, parameter_values=parameter_values, C_rate = C_Rates[i])
    sim.solve([0,3600])
    nsoc = sim.solution["Negative electrode SOC"].entries
    temp = sim.solution["Negative electrode temperature [K]"].entries[-1]
    voltage = sim.solution["Terminal voltage [V]"].entries
    
    for j in range(len(nsoc)):
        C_Rate.append(C_Rates[i])
        NSOC.append(nsoc[j])
        Temp.append(temp[j])
        Voltage.append(voltage[j])

In [5]:
# creating a dataframe and storing the results

import pandas as pd

df = pd.DataFrame()
df['C_Rate'] = C_Rate
df['NSOC'] = NSOC
df['Temp'] = Temp
df['Voltage'] = Voltage
# storing result into spmt.csv file
df.to_csv(r"spmt_data.csv")
df.head()

,C_Rate,NSOC,Temp,Voltage
0,0.1,0.800000,298.150000,3.841727
1,0.1,0.799397,298.153985,3.840125
2,0.1,0.798793,298.158054,3.839417
3,0.1,0.798190,298.162147,3.838888
4,0.1,0.797586,298.166249,3.838417


In [12]:
from sklearn.model_selection import train_test_split
data = pd.read_csv(r"spmt_data.csv")
df = pd.DataFrame(data)

# Split data
x = df.drop(df.columns[[0,4]], axis = 1)
y = df.drop(df.columns[[0,1,2,3]], axis = 1)
x_train, x_test, y_train, y_test = train_test_split(x,y,test_size = 0.2, random_state = 42)
print(x)
print(y)

     C_Rate      NSOC        Temp
0       0.1  0.800000  298.150000
1       0.1  0.799397  298.153985
2       0.1  0.798793  298.158054
3       0.1  0.798190  298.162147
4       0.1  0.797586  298.166249
..      ...       ...         ...
614    10.0  0.437950  335.525786
615    10.0  0.377609  339.655741
616    10.0  0.317267  343.603074
617    10.0  0.256926  347.712462
618    10.0  0.227130  351.000544

[619 rows x 3 columns]
      Voltage
0    3.841727
1    3.840125
2    3.839417
3    3.838888
4    3.838417
..        ...
614  3.523391
615  3.530209
616  3.532404
617  3.447120
618  3.105019

[619 rows x 1 columns]


In [13]:
import tensorflow as tf
from tensorflow import keras

# defining model
model = keras.Sequential([
    keras.layers.Dense(620, activation = 'relu', input_dim = 3),
    keras.layers.Dense(32, activation = 'relu'),
    keras.layers.Dense(1, activation = 'linear')
])

# opt = keras.optimizers.Adam(learning_rate=0.01)

# compiling model
model.compile(optimizer = 'adam',
             loss = 'mean_squared_error',
             metrics = ['mean_squared_error'])

# fitting model
model.fit(x_train,y_train, epochs = 100)


Epoch 1/100
16/16 [==============================] - 0s 799us/step - loss: 222.6684 - mean_squared_error: 222.6684
Epoch 2/100
16/16 [==============================] - 0s 1ms/step - loss: 17.6673 - mean_squared_error: 17.6673
Epoch 3/100
16/16 [==============================] - 0s 931us/step - loss: 6.2687 - mean_squared_error: 6.2687
Epoch 4/100
16/16 [==============================] - 0s 653us/step - loss: 1.2206 - mean_squared_error: 1.2206
Epoch 5/100
16/16 [==============================] - 0s 711us/step - loss: 0.2405 - mean_squared_error: 0.2405
Epoch 6/100
16/16 [==============================] - 0s 665us/step - loss: 0.1183 - mean_squared_error: 0.1183
Epoch 7/100
16/16 [==============================] - 0s 731us/step - loss: 0.0458 - mean_squared_error: 0.0458
Epoch 8/100
16/16 [==============================] - 0s 731us/step - loss: 0.0352 - mean_squared_error: 0.0352
Epoch 9/100
16/16 [==============================] - 0s 719us/step - loss: 0.0311 - mean_squared_error: 0.03

16/16 [==============================] - 0s 729us/step - loss: 0.0189 - mean_squared_error: 0.0189
Epoch 75/100
16/16 [==============================] - 0s 1ms/step - loss: 0.0389 - mean_squared_error: 0.0389
Epoch 76/100
16/16 [==============================] - 0s 925us/step - loss: 0.0171 - mean_squared_error: 0.0171
Epoch 77/100
16/16 [==============================] - 0s 777us/step - loss: 0.0090 - mean_squared_error: 0.0090
Epoch 78/100
16/16 [==============================] - 0s 785us/step - loss: 0.0154 - mean_squared_error: 0.0154
Epoch 79/100
16/16 [==============================] - 0s 653us/step - loss: 0.0192 - mean_squared_error: 0.0192
Epoch 80/100
16/16 [==============================] - 0s 532us/step - loss: 0.0248 - mean_squared_error: 0.0248
Epoch 81/100
16/16 [==============================] - 0s 651us/step - loss: 0.0324 - mean_squared_error: 0.0324
Epoch 82/100
16/16 [==============================] - 0s 655us/step - loss: 0.0215 - mean_squared_error: 0.0215
Epoch 8

In [14]:
score = model.evaluate(x_test, y_test)
y_predicted = model.predict(x)
df['Predicted_Voltage'] = y_predicted
df.to_csv(r"data.csv")
df.head(620)

4/4 [==============================] - 0s 662us/step - loss: 0.0186 - mean_squared_error: 0.0186


,Unnamed: 0,C_Rate,NSOC,Temp,Voltage,Predicted_Voltage
0,0,0.1,0.800000,298.150000,3.841727,3.677105
1,1,0.1,0.799397,298.153985,3.840125,3.676908
2,2,0.1,0.798793,298.158054,3.839417,3.676718
3,3,0.1,0.798190,298.162147,3.838888,3.676524
4,4,0.1,0.797586,298.166249,3.838417,3.676330
...,...,...,...,...,...,...
614,614,10.0,0.437950,335.525786,3.523391,3.253052
615,615,10.0,0.377609,339.655741,3.530209,3.274852
616,616,10.0,0.317267,343.603074,3.532404,3.294625
617,617,10.0,0.256926,347.712462,3.447120,3.316193


## NSOC

In [79]:
from sklearn.model_selection import train_test_split
data = pd.read_csv(r"data.csv")
df = pd.DataFrame(data)

# Split data
x = df.drop(df.columns[[0,1,3,5]], axis = 1)
y = df.drop(df.columns[[0,1,2,4,5,6]], axis = 1)
x_train, x_test, y_train, y_test = train_test_split(x,y,test_size = 0.2, random_state = 0)
print(x)
print(y)

     C_Rate        Temp  Predicted_Voltage
0       0.1  298.150000           3.677105
1       0.1  298.153985           3.676908
2       0.1  298.158054           3.676718
3       0.1  298.162147           3.676524
4       0.1  298.166249           3.676330
..      ...         ...                ...
614    10.0  335.525786           3.253051
615    10.0  339.655741           3.274852
616    10.0  343.603074           3.294625
617    10.0  347.712462           3.316193
618    10.0  351.000544           3.340777

[619 rows x 3 columns]
         NSOC
0    0.800000
1    0.799397
2    0.798793
3    0.798190
4    0.797586
..        ...
614  0.437950
615  0.377609
616  0.317267
617  0.256926
618  0.227130

[619 rows x 1 columns]


In [80]:
import tensorflow as tf
from tensorflow import keras

# defining model
model = keras.Sequential([
    keras.layers.Dense(619, activation = 'relu', input_dim = 3),
    keras.layers.Dense(62, activation = 'relu'),
    keras.layers.Dense(8, activation = 'relu'),
    keras.layers.Dense(1, activation = 'linear')
])

# opt = keras.optimizers.Adam(learning_rate=0.01)

# compiling model
model.compile(optimizer = 'adam',
             loss = 'mean_squared_error',
             metrics = ['mean_squared_error'])

# fitting model
model.fit(x_train,y_train, epochs = 100)


Epoch 1/100
16/16 [==============================] - 0s 964us/step - loss: 19.1153 - mean_squared_error: 19.1153
Epoch 2/100
16/16 [==============================] - 0s 930us/step - loss: 2.1423 - mean_squared_error: 2.1423
Epoch 3/100
16/16 [==============================] - 0s 918us/step - loss: 0.4534 - mean_squared_error: 0.4534
Epoch 4/100
16/16 [==============================] - 0s 837us/step - loss: 0.1515 - mean_squared_error: 0.1515
Epoch 5/100
16/16 [==============================] - 0s 836us/step - loss: 0.0464 - mean_squared_error: 0.0464
Epoch 6/100
16/16 [==============================] - 0s 919us/step - loss: 0.0349 - mean_squared_error: 0.0349
Epoch 7/100
16/16 [==============================] - 0s 840us/step - loss: 0.0369 - mean_squared_error: 0.0369
Epoch 8/100
16/16 [==============================] - 0s 901us/step - loss: 0.0352 - mean_squared_error: 0.0352
Epoch 9/100
16/16 [==============================] - 0s 974us/step - loss: 0.0311 - mean_squared_error: 0.0311

16/16 [==============================] - 0s 798us/step - loss: 0.0473 - mean_squared_error: 0.0473
Epoch 75/100
16/16 [==============================] - 0s 798us/step - loss: 0.0359 - mean_squared_error: 0.0359
Epoch 76/100
16/16 [==============================] - 0s 864us/step - loss: 0.0653 - mean_squared_error: 0.0653
Epoch 77/100
16/16 [==============================] - 0s 766us/step - loss: 0.0470 - mean_squared_error: 0.0470
Epoch 78/100
16/16 [==============================] - 0s 716us/step - loss: 0.0441 - mean_squared_error: 0.0441
Epoch 79/100
16/16 [==============================] - 0s 787us/step - loss: 0.0279 - mean_squared_error: 0.0279
Epoch 80/100
16/16 [==============================] - 0s 864us/step - loss: 0.0552 - mean_squared_error: 0.0552
Epoch 81/100
16/16 [==============================] - 0s 976us/step - loss: 0.0361 - mean_squared_error: 0.0361
Epoch 82/100
16/16 [==============================] - 0s 997us/step - loss: 0.0713 - mean_squared_error: 0.0713
Epoch

In [81]:
df2 = df
df2 = df.drop(df.columns[[0,1]], axis = 1)

In [82]:
score = model.evaluate(x_test, y_test)
print(score)

4/4 [==============================] - 0s 998us/step - loss: 0.0243 - mean_squared_error: 0.0243
[0.02429179474711418, 0.02429179474711418]


In [83]:
nsoc_predicted = model.predict(x)
df2['Predicted_NSOC'] = nsoc_predicted
df2.head(620)

,C_Rate,NSOC,Temp,Voltage,Predicted_Voltage,Predicted_NSOC
0,0.1,0.800000,298.150000,3.841727,3.677105,0.657746
1,0.1,0.799397,298.153985,3.840125,3.676908,0.657742
2,0.1,0.798793,298.158054,3.839417,3.676718,0.657739
3,0.1,0.798190,298.162147,3.838888,3.676524,0.657733
4,0.1,0.797586,298.166249,3.838417,3.676330,0.657729
...,...,...,...,...,...,...
614,10.0,0.437950,335.525786,3.523391,3.253051,0.385276
615,10.0,0.377609,339.655741,3.530209,3.274852,0.391659
616,10.0,0.317267,343.603074,3.532404,3.294625,0.397705
617,10.0,0.256926,347.712462,3.447120,3.316193,0.404051


In [84]:
# defining RMSE
import math

def RMSE(true_value, model_value):
    res = (true_value-model_value)**2
    return math.sqrt(res)

In [85]:
nsoc = df2['NSOC']
predicted_nsoc = df2['Predicted_NSOC']

rmse = []
n = len(nsoc)
for i in range(n):
    temp = RMSE(nsoc[i],predicted_nsoc[i])
    rmse.append(temp)

In [86]:
df2['NSOC_RMSE'] = rmse

In [87]:
df2.head(620)

,C_Rate,NSOC,Temp,Voltage,Predicted_Voltage,Predicted_NSOC,NSOC_RMSE
0,0.1,0.800000,298.150000,3.841727,3.677105,0.657746,0.142254
1,0.1,0.799397,298.153985,3.840125,3.676908,0.657742,0.141655
2,0.1,0.798793,298.158054,3.839417,3.676718,0.657739,0.141054
3,0.1,0.798190,298.162147,3.838888,3.676524,0.657733,0.140456
4,0.1,0.797586,298.166249,3.838417,3.676330,0.657729,0.139857
...,...,...,...,...,...,...,...
614,10.0,0.437950,335.525786,3.523391,3.253051,0.385276,0.052674
615,10.0,0.377609,339.655741,3.530209,3.274852,0.391659,0.014050
616,10.0,0.317267,343.603074,3.532404,3.294625,0.397705,0.080438
617,10.0,0.256926,347.712462,3.447120,3.316193,0.404051,0.147125


In [88]:
def ERROR(value):
    sum = 0
    for v in value:
        sum+=v
    return sum/len(value)

In [89]:
error = ERROR(rmse)
error

0.12982195338421254

## Temp

In [90]:
from sklearn.model_selection import train_test_split
data = pd.read_csv(r"data.csv")
df = pd.DataFrame(data)

# Split data
x = df.drop(df.columns[[0,1,4,5]], axis = 1)
y = df.drop(df.columns[[0,1,2,3,5,6]], axis = 1)
x_train, x_test, y_train, y_test = train_test_split(x,y,test_size = 0.2, random_state = 0)
print(x)
print(y)

     C_Rate      NSOC  Predicted_Voltage
0       0.1  0.800000           3.677105
1       0.1  0.799397           3.676908
2       0.1  0.798793           3.676718
3       0.1  0.798190           3.676524
4       0.1  0.797586           3.676330
..      ...       ...                ...
614    10.0  0.437950           3.253051
615    10.0  0.377609           3.274852
616    10.0  0.317267           3.294625
617    10.0  0.256926           3.316193
618    10.0  0.227130           3.340777

[619 rows x 3 columns]
           Temp
0    298.150000
1    298.153985
2    298.158054
3    298.162147
4    298.166249
..          ...
614  335.525786
615  339.655741
616  343.603074
617  347.712462
618  351.000544

[619 rows x 1 columns]


In [91]:
# defining model
model = keras.Sequential([
    keras.layers.Dense(619, activation = 'relu', input_dim = 3),
    keras.layers.Dense(62, activation = 'relu'),
    keras.layers.Dense(8, activation = 'relu'),
    keras.layers.Dense(1, activation = 'linear')
])

# opt = keras.optimizers.Adam(learning_rate=0.01)

# compiling model
model.compile(optimizer = 'adam',
             loss = 'mean_squared_error',
             metrics = ['mean_squared_error'])

# fitting model
model.fit(x_train,y_train, epochs = 100)

Epoch 1/100
16/16 [==============================] - 0s 1ms/step - loss: 92657.5666 - mean_squared_error: 92657.5666
Epoch 2/100
16/16 [==============================] - 0s 1ms/step - loss: 87391.2528 - mean_squared_error: 87391.2528
Epoch 3/100
16/16 [==============================] - 0s 931us/step - loss: 74159.2390 - mean_squared_error: 74159.2390
Epoch 4/100
16/16 [==============================] - 0s 1ms/step - loss: 47186.3127 - mean_squared_error: 47186.3127
Epoch 5/100
16/16 [==============================] - 0s 1ms/step - loss: 14851.9818 - mean_squared_error: 14851.9818
Epoch 6/100
16/16 [==============================] - 0s 1ms/step - loss: 4747.1379 - mean_squared_error: 4747.1379
Epoch 7/100
16/16 [==============================] - 0s 882us/step - loss: 2239.2577 - mean_squared_error: 2239.2577
Epoch 8/100
16/16 [==============================] - 0s 1ms/step - loss: 1046.8331 - mean_squared_error: 1046.8331
Epoch 9/100
16/16 [==============================] - 0s 1ms/step -

Epoch 73/100
16/16 [==============================] - 0s 893us/step - loss: 7.0158 - mean_squared_error: 7.0158
Epoch 74/100
16/16 [==============================] - 0s 987us/step - loss: 6.6325 - mean_squared_error: 6.6325
Epoch 75/100
16/16 [==============================] - 0s 850us/step - loss: 5.7603 - mean_squared_error: 5.7603
Epoch 76/100
16/16 [==============================] - 0s 893us/step - loss: 4.8884 - mean_squared_error: 4.8884
Epoch 77/100
16/16 [==============================] - 0s 784us/step - loss: 4.6224 - mean_squared_error: 4.6224
Epoch 78/100
16/16 [==============================] - 0s 700us/step - loss: 4.1400 - mean_squared_error: 4.1400
Epoch 79/100
16/16 [==============================] - 0s 721us/step - loss: 3.3774 - mean_squared_error: 3.3774
Epoch 80/100
16/16 [==============================] - 0s 695us/step - loss: 2.8772 - mean_squared_error: 2.8772
Epoch 81/100
16/16 [==============================] - 0s 719us/step - loss: 2.8071 - mean_squared_error:

In [92]:
df2 = df
df2 = df.drop(df.columns[[0,1]], axis = 1)

In [93]:
score = model.evaluate(x_test, y_test)
print(score)

4/4 [==============================] - 0s 838us/step - loss: 0.3361 - mean_squared_error: 0.3361
[0.33611735701560974, 0.33611735701560974]


In [94]:
temp_predicted = model.predict(x)
df2['Predicted_Temp'] = temp_predicted

In [95]:
temp = df2['Temp']
predicted_temp = df2['Predicted_Temp']

rmse = []
n = len(nsoc)
for i in range(n):
    t = RMSE(temp[i],predicted_temp[i])
    rmse.append(t)

In [96]:
df2['Temp_RMSE'] = rmse
df2.head(620)

,C_Rate,NSOC,Temp,Voltage,Predicted_Voltage,Predicted_Temp,Temp_RMSE
0,0.1,0.800000,298.150000,3.841727,3.677105,297.726318,0.423682
1,0.1,0.799397,298.153985,3.840125,3.676908,297.727905,0.426080
2,0.1,0.798793,298.158054,3.839417,3.676718,297.730042,0.428012
3,0.1,0.798190,298.162147,3.838888,3.676524,297.731903,0.430244
4,0.1,0.797586,298.166249,3.838417,3.676330,297.733704,0.432546
...,...,...,...,...,...,...,...
614,10.0,0.437950,335.525786,3.523391,3.253051,334.335754,1.190031
615,10.0,0.377609,339.655741,3.530209,3.274852,338.058014,1.597727
616,10.0,0.317267,343.603074,3.532404,3.294625,341.631104,1.971970
617,10.0,0.256926,347.712462,3.447120,3.316193,345.336243,2.376219


In [97]:
error = ERROR(rmse)
error

0.4701257561376197